# Imports 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import imageio
import os
import ast
import sys

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.nn.parameter import Parameter
import torchvision
import numpy as np

from torchvision import transforms


val2017_dir = '/content/drive/MyDrive/Thesis/Data/COCO/val2017'
train2017_dir = '/content/drive/MyDrive/Thesis/Data/COCO/train2017'
anno2017_dir = '/content/drive/MyDrive/Thesis/Data/COCO/annotations_trainval2017'

val2017 = 'val2017'
train2017 = 'train2017'
annFile = '/content/drive/MyDrive/Thesis/Data/COCO/annotations/instances_val2017.json'

COCOtoCIFAR = {'airplane':'plane', 'car':'car', 'bird':'bird', 'cat':'cat', 'dog':'dog', 'horse':'horse', 'boat':'ship', 'truck':'truck'} 

In [ ]:
from pycocotools.coco import COCO
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

coco=COCO(annFile)




# for i in range(len(annos)):
# 	entity_id = annos[i]["category_id"]
# 	entity = coco.loadCats(entity_id)[0]["name"]
# 	print(True if (entity in classes) else False)
# 	print("{}: {}".format(i, entity))
 
# print([coco.loadCats(c["category_id"])[0]["name"] for c in annos])

# imgIds = coco.getImgIds(imgIds = [id])
# img = coco.loadImgs(imgIds)[0]

# I = io.imread(img['coco_url'])
# plt.axis('off')
# plt.imshow(I)
# plt.show()

def CIFARCheck(imgId, maxCat=5):
	annIds = coco.getAnnIds(imgIds=imgId)
	annos = coco.loadAnns(annIds)
	cats = [coco.loadCats(c["category_id"])[0]["name"] for c in annos]
	if len(cats) < maxCat and bool(set(cats) & set(COCOtoCIFAR.keys())): 
		return list(set(cats) & set(COCOtoCIFAR.keys()))
	else:
		return False



loading annotations into memory...
Done (t=3.14s)
creating index...
index created!


# COCO to NSD

In [ ]:
nsd = pd.read_csv('/content/drive/MyDrive/Thesis/Data/NSD/nsddata/experiments/nsd/nsd_stim_info_merged.csv', index_col=0)

nsd_cifar = pd.DataFrame(columns=nsd.columns)
nsd_cifar['class'] = np.nan

found = []

for index, row in nsd.iterrows():
    cats = CIFARCheck(int(row['cocoId']), 2)
    if cats is not False:
        cats = [COCOtoCIFAR[c] for c in cats]
        row['class'] = cats[0]
        nsd_cifar = pd.concat([nsd_cifar,row.to_frame().T], ignore_index=True)

        for cat in cats:
            if cat not in found:
                found.append(cat)
                

In [ ]:
nsd_cifar['class'].value_counts()

plane    33
bird     26
cat      17
horse    14
dog       9
truck     6
ship      6
car       3
Name: class, dtype: int64

In [ ]:
nsd_cifar.to_csv('/content/drive/MyDrive/Thesis/Data/NSD/nsd_coco_cifar_pre.csv',index=False)